In [55]:
import pandas as pd
import pypfopt
from pypfopt import EfficientFrontier, plotting
import matplotlib.pyplot as plt
pypfopt.__version__

'1.5.5'

In [39]:
# data importation
r = pd.read_csv('return.CSV', sep=',', header=None, index_col=0)
r = r.squeeze()
cov = pd.read_csv('cov.CSV', sep=',', header=0, index_col=0)



In [58]:
cov

,us_bond,us_equity,developed_equity,emerging_equity,hedge_fund,reit,commodity,gold,btc
Ticker,,,,,,,,,
us_bond,0.0033,0.0040,0.0028,0.0044,0.0012,0.0043,-0.0049,0.0041,-0.0737
us_equity,0.0040,0.0248,0.0203,0.0183,0.0090,0.0153,0.0042,0.0004,0.9619
developed_equity,0.0028,0.0203,0.0207,0.0195,0.0079,0.0115,0.0069,0.0023,0.8156
emerging_equity,0.0044,0.0183,0.0195,0.0261,0.0081,0.0116,0.0041,0.0114,-0.0785
hedge_fund,0.0012,0.0090,0.0079,0.0081,0.0038,0.0047,0.0028,0.0018,0.2155
reit,0.0043,0.0153,0.0115,0.0116,0.0047,0.0245,0.0068,0.0007,-0.2240
commodity,-0.0049,0.0042,0.0069,0.0041,0.0028,0.0068,0.0475,0.0016,0.0124
gold,0.0041,0.0004,0.0023,0.0114,0.0018,0.0007,0.0016,0.0177,-1.1228
btc,-0.0737,0.9619,0.8156,-0.0785,0.2155,-0.2240,0.0124,-1.1228,224.0111


In [57]:
r

0
us_bond             0.020350
us_equity           0.119000
developed_equity    0.051706
emerging_equity     0.028604
hedge_fund          0.041399
reit                0.069545
commodity          -0.003898
gold                0.030725
btc                 7.298820
Name: 1, dtype: float64

In [67]:
ef = EfficientFrontier(r, cov)


In [68]:
ef.max_sharpe()

OrderedDict([('us_bond', 0.0),
             ('us_equity', 0.1677458673674235),
             ('developed_equity', 0.0),
             ('emerging_equity', 0.0),
             ('hedge_fund', 0.0),
             ('reit', 0.2682486566646236),
             ('commodity', 0.0),
             ('gold', 0.556519106513134),
             ('btc', 0.0074863694548188)])

In [60]:
weights = ef.clean_weights()

In [61]:
weights

OrderedDict([('us_bond', 0.19999),
             ('us_equity', 0.0),
             ('developed_equity', 0.15),
             ('emerging_equity', 0.0),
             ('hedge_fund', 0.0),
             ('reit', 0.0),
             ('commodity', 0.0),
             ('gold', 0.0),
             ('btc', 0.64999)])

In [69]:
ef.add_constraint(lambda w: w[0] >= 0.2)
ef.add_constraint(lambda w: w[2] == 0.15)
ef.add_constraint(lambda w: w[3] + w[4] <= 0.10)

fig, ax = plt.subplots()
plotting.plot_efficient_frontier(ef, ax=ax, show_assets=True)
plt.show()

InstantiationError: Adding constraints to an already solved problem might have unintended consequences. A new instance should be created for the new set of constraints.